In [13]:
import requests
import json
import pprint

In [36]:
def runGraphQLQuery(url,query, variables):
    headers = {"content-type":"application/json"}
    try:
        #results = requests.post(url = url, json={"query":query, "headers":headers}, verify = False)
        results = requests.post(url = url, json={"query":query, "headers":headers, "variables":variables})
        results = results.json()
    except requests.exceptions.HTTPError as exception:
        print(exception)
    return results

In [47]:
def parseFields(results):
    datafields = []
    fieldlist = results["data"]["__type"]["fields"]
    if fieldlist is not None:
        for field in fieldlist:
            datafields.append(field["name"])
    return datafields

In [38]:
cds_url = "https://dataservice.datacommons.cancer.gov/v1/graphql/"

In [39]:
schema_query = """
{
  __schema{
    types{
      name
    }
  }
}"""

In [40]:
namelist = runGraphQLQuery(cds_url, schema_query, None)

In [41]:
namelist = namelist["data"]["__schema"]["types"]

In [42]:
nodelist = []
for name in namelist:
    nodelist.append(name["name"])

In [43]:
fieldquery = """
query FieldLister($node:String!){
  __type(name:$node){
    name
    fields{
      name
      type{
        name
        kind
      }
    }
  }
}"""

In [49]:
finaldata = {}
for node in nodelist:
    variables = {"node":node}
    results = runGraphQLQuery(cds_url, fieldquery, variables)
    #pprint.pprint(results)
    fieldlist = parseFields(results)
    finaldata[node] = fieldlist
    

In [52]:
for key, value in finaldata.items():
    if len(value) >=1:
        pprint.pprint(key)
        pprint.pprint(value)
#pprint.pprint(finaldata)

'FileInfo'
['subject_id',
 'file_name',
 'file_type',
 'association',
 'file_description',
 'file_size',
 'file_id',
 'md5sum']
'FileOverview'
['study_acronym',
 'accesses',
 'phs_accession',
 'subject_id',
 'sample_id',
 'experimental_strategy',
 'gender',
 'analyte_type',
 'is_tumor',
 'file_name',
 'file_type',
 'file_size',
 'file_id',
 'md5sum']
'GS_About'
['page', 'title', 'type', 'text']
'GS_File'
['type',
 'subject_id',
 'sample_id',
 'file_id',
 'file_name',
 'file_type',
 'accesses',
 'acl',
 'experimental_strategies',
 'instrument_models',
 'library_layouts',
 'library_selections',
 'library_sources',
 'library_strategies',
 'platforms',
 'reference_genome_assemblies',
 'sites']
'GS_Model'
['type',
 'node_name',
 'property_name',
 'property_description',
 'property_required',
 'property_type',
 'value',
 'highlight']
'GS_Program'
['program_name',
 'program_short_description',
 'program_full_description',
 'program_external_url',
 'program_sort_order',
 'type']
'GS_Sample'
['